剪绳子，把一根长度为n>1的绳子剪成m>1段，求各段长度累乘值最大的剪切方案。此题难点在于找到最开始的几个特殊情况，即什么长度下不剪的总长度要小于剪开的长度累乘？

In [1]:
# 设解为f(n)，列出f(n)<n的特殊情况
# n=0，f(0)无解；n=1，f(1)无解；n=2，f(2)=1*1=1；n=3，f(3)=1*2=2；n=4，f(4)=4=n
# n>3时，DP递推式为f(n)=max(f(i)*f(n-i))，i为切割点


def MaxProductCut(n):
    if n < 2:
        return None
    if n == 2:
        return 1
    if n == 3:
        return 2

    f_n = [0]*(n+1)    # 存储能用于递推式的f(n)的数组
    f_n[1] = 1
    f_n[2] = 2
    f_n[3] = 3

    for i in range(4, n+1):    # 从小到大求出所有最优解
        max_product = 0
        for cut_point in range(1, i//2+1):    # 尝试所有切分点，切割长度至少为1
            cur_product = f_n[cut_point]*f_n[i-cut_point]
            if cur_product > max_product:
                max_product = cur_product
        f_n[i] = max_product
    return f_n[-1]

矩阵中得到最大和的路线。一个矩阵，每个单元格上均有一个数字，从左上角开始，每次只能往右或往下，求一条能得到最大和的路径。

In [2]:
# 思路：动态规划，维护一个最大价值矩阵，该矩阵中每个元素的值均由max(up,left)+cur计算得出
# 手动模拟时发现，在计算时最多只用到了上一行的数据，所以在空间上可以进一步优化，不需要维护个完整的矩阵


def getMaxSum(values, rows, cols):
    '''
    values:数字序列
    rows,cols:矩阵行列数
    '''
    sum_mat = [[0 for _ in range(cols)] for _ in range(rows)]    # sum矩阵

    # 从左往右逐行扫描
    for row in range(rows):
        for col in range(cols):
            up = sum_mat[row-1][col] if row > 0 else 0    # 如果是第一行，上格子值设为0
            left = sum_mat[row][col-1] if col > 0 else 0    # 如果是第一列，左格子值设为0

            sum_mat[row][col] = max(up, left)+values[row*cols+col]

    return sum_mat[-1][-1]

非重复最长子串。给定一个字符串，求其中不含重复字母的最长子串长度。

In [3]:
# 思路：为26个字母维护一个记录出现位置的数组，扫描字符串
# 如果上次出现位置在当前子串之前，则将该字母纳入子串
# 如果该字母已出现在子串内，则删掉重复字母及之前的部分，再将该字母纳入子串


def LongestSubstringWithoutDup(s):
    max_len = 0
    cur_len = 0
    ch_appear_pos = [-1]*26    # 记录每个字母最近一次出现的位置

    for idx in (range(len(s))):    # 扫描字符串
        pre_pos = ch_appear_pos[ord(s[idx])-ord('a')]    # 当前字母上次出现的位置

        if pre_pos < 0 or idx-pre_pos > cur_len:    # 如果上次出现的距离比当前保存的距离还远
            cur_len += 1    # 则把该字母加入子串中
        else:    # 如果该字母是子串中已出现的字母，则需要缩短子串
            if cur_len > max_len:
                max_len = cur_len
            cur_len = idx-pre_pos    # 去掉该字母上次出现位置及之前的部分
        ch_appear_pos[ord(s[idx])-ord('a')] = idx    # 更新该字母出现位置

    if cur_len > max_len:
        max_len = cur_len

    return max_len

n个骰子点数和的概率。

In [4]:
# 思路：令f(n,s)为n个骰子和为s的组合数，则
# f(n,s)=f(n-1,s-1)+f(n-1,s-2)+f(n-1,s-3)+f(n-1,s-4)+f(n-1,s-5)+f(n-1,s-6)
# 可见f(n,*)只与f(n-1,*)时的状态有关，设立两个数组，分别轮番表示f(n-1,*)与f(n,*)时的状态

def P(n):
    if n<1:
        return 0
    
    max_value=6    # 预定义骰子的最大点数
    
    # n个骰子的点数范围为n-6n，把数组长度设为6n，非法位置设为0
    arr1=[0 for _ in range(max_value*n+1)]
    arr2=[0 for _ in range(max_value*n+1)]
    arr=[arr1,arr2]
    flag=0    # 用于在两数组中进行状态轮切
    
    # 初始状态f(1,1)=f(1,2)=...=f(1,6)=1
    for i in range(1,max_value+1):
        arr[flag][i]=1
    
    for k in range(2,n+1):    # 逐个增加骰子
        for val in range(0,k-1):    # 小于骰子数的面值概率都为0
            arr[1-flag][val]=0
            
        for val in range(k,max_value*k+1):
            arr[1-flag][val]=0    # 初始值设为0
            
            j=1    # 逐个加上上一状态时-1,-2,...,-6的组合数
            while j<=val and j<=max_value:
                arr[1-flag][val]+=arr[flag][val-j]
                j+=1
        
        flag=1-flag    # 转换状态
        
    tol_comb=pow(max_value,n)
    for i in range(n,max_value*n+1):
        print(arr[flag][i]/tol_comb)

Ugly Number II. 丑数。除1之外，只有因子2，3，5的数称为丑数。查找第n个丑数。

In [5]:
# 思路：1是第一个丑数，也可看作是所有丑数的基数
# 任何丑数都是由某个丑数乘以2、或乘以3、或乘以5得到
# 如2，3，5就是分别由1乘2，乘3，乘5得到
# 任何丑数都有*2，*3和*5的机会，所以设立三个指针
# 三个指针在已有的丑数数组上逐位移动

def nthUglyNumber(n):
    ugly_nums=[1]
    if n==1:
        return ugly_nums[-1]
    
    # 乘2乘3乘5的机会指针
    mutil2_idx=mutil3_idx=mutil5_idx=0
    
    cnt=1    # 计数
    while cnt<n:
        cur_ugly=min(ugly_nums[mutil2_idx]*2,ugly_nums[mutil3_idx]*3,ugly_nums[mutil5_idx]*5)
        
        if cur_ugly>ugly_nums[-1]:
            ugly_nums.append(cur_ugly)
            cnt+=1
        
        if cur_ugly==ugly_nums[mutil2_idx]*2:
            mutil2_idx+=1
        elif cur_ugly==ugly_nums[mutil3_idx]*3:
            mutil3_idx+=1
        else:
            mutil5_idx+=1
            
    return ugly_nums[-1]

(没搞懂！)[歌单方案数](https://www.nowcoder.com/questionTerminal/f3ab6fe72af34b71a2fd1d83304cbbb3?orderByHotValue=1&page=1&onlyReference=false)，手头有X首A分钟的歌，还有Y首B分钟的歌，要组成一个总长K分钟的歌单，有多少种方案。$K\le1000$，$X,Y\le100$

In [17]:
# 思路：动态规划，设index为歌单内歌的数量，column为歌单长度
# 难点在于状态转移方程，C(n,k)= C(n-1,k) + C(n-1,k-1)
# n表示歌总数，k表示选出来的歌的数量，题目中k是以时长表示的
# 所以该题的状态方程为
# dp[num_song][total_duration]=dp[num_song-1][total_duration]+dp[num_song-1][total_duration-len_song]


def solution(A, X, B, Y, K):
    dp = [[0 for _ in range(K+1)] for _ in range(X+Y+1)]
    lens = [0]*(X+Y+1)

    dp[0][0] = 1    # 0首歌组成0分钟的歌单，一种方案
    for i in range(1, X+1):
        lens[i] = A
    for i in range(X+1, X+Y+1):
        lens[i] = B

    for num_song in range(1, X+Y+1):
        for total_duration in range(0, K+1):
            dp[num_song][total_duration] = dp[num_song-1][total_duration] if total_duration < lens[num_song] \
                else dp[num_song - 1][total_duration]+dp[num_song-1][total_duration-lens[num_song]]

    return dp[X+Y][K]


solution(2, 3, 3, 3, 5)

9

[Fibonacci Number](https://leetcode.com/problems/fibonacci-number/). 

In [6]:
# 思路：空间换时间


def fib(N: int) -> int:
    if N < 2:
        return N

    pre = 0
    cur = 1

    for _ in range(2, N+1):
        pre += cur    # 前两个数加起来，覆盖pre
        pre, cur = cur, pre    # 交换两值

    return cur


fib(4)

3